# 3. MalRNN 코드 작성

* 필요한 package 설치 확인 및 불러오기

In [2]:
# !pip install numpy
# !pip install torch

In [ ]:
import numpy as np
import random
import torch
import torch.nn.functional as F

## 1. 추출된 Stream data 에서의 시퀀스 추출 과정 구축
* 추출된 stream data(.csv)에서 필요한 byte stream data 추출

In [1]:
def parse_data(data_path, chunk_len, malconv_min_len, len_limit=100000):
    """
    :param str data_path: stream data csv파일의 위치
    :param int chunk_len: benign byte stream sampling 길이(정상데이터는 최소 chunk_len 이상이어야 함)
    :param int malconv_min_len: malconv 탐지를 위한 최소 길이(악성데이터는 최소 malconv_min_len 이상이어야 함)
    :param int len_limit: 시스템 과부하 방지 목적 길이 제한
    :return 정상/악성 byte stream data list
    """
    
    ### Byte Stream 데이터 추출을 위한 함수 만들기 ###
    # 1. data_path에 위치한 파일 열기
    # 2. 정상/악성 데이터를 저장할 list 생성
    # 3. len_limit 이내의 데이터에서 추출
    # 4. csv 내의 Stream 이후만 추출하기 위한 "]"의 위치 찾은후 이후 데이터만 추출
    # 5. 추출된 데이터의 개행문자 제거 및 전처리
    # 6. 전처리된 데이터의 int 변환 및 list 삽입
    # 7. 정상/악성 여부에 따른 최소 길이 확인 및 list 삽입
    
    with open(data_path, "r", encoding="cp949") as data:
        benign_data = []
        critical_data = []

        for line in data.readlines():
            try:
                if len(line) < len_limit:
                    look = line.rfind("]")
                    line = line[look+2 : ]
                    line = line.replace("\n", "")
                    if line.startswith(","):
                        line = line[1:]
                    line = line.split(",")
                    line = [int(x,0) for x in line]

                    if line[1] == 0 and len(line[4:]) > chunk_len:
                        benign_data.append(line[4:])
                    elif line[1] == 1 and len(line[4:]) > malconv_min_len:
                        critical_data.append(line[4:])
            except:
                pass
    
    return benign_data, critical_data

## 2. 생성된 byte stream 악성 여부 확인

In [ ]:
def eval_detection(malconv, gen_bytes):
    """
    :param nn.Module malconv: malconv 모델
    :param list gen_bytes: 생성된 byte stream 
    """
    with torch.no_grad():
        gen_bytes = torch.from_numpy(np.frombuffer(gen_bytes, dtype=np.uint8)[np.newaxis, :])
        malconv_output = F.softmax(malconv(gen_bytes), dim=-1).detach().numpy()[0,1]
        return malconv_output

## 3. 정상 데이터에 대한 sampling

In [3]:
def create_benign_sample(benign_stream, chunk_len, batch_size, device):
    """
    :param list benign_stream: 정상 byte stream
    :param int chunk_len: sampling 길이
    :param int batch_size: 한번 학습에 이용할 byte 갯수
    :param device: 학습 device(CPU/GPU)에 따른 할당
    :return input_stream, target_stream
    """
    ### 정상 데이터 sampling 만들기 ###
    # 1. input_stream(학습에 사용), target_stream(loss 산정 시 사용) 선언 (batch_size * chunk_len)
    # 2. batch size 만큼의 for loop 생성
    # 3. chunk_len을 고려한 임의의 start index 선정
    # 4. chunk_len을 고려한 end_index 계산
    # 5. benign stream data에서 sampling
    # 6. input_stream과 target_stream 저장
    # 7. 저장된 input_stream, target_stream device 할당
    
    input_stream = torch.LongTensor(batch_size, chunk_len)
    target_stream = torch.LongTensor(batch_size, chunk_len)
    for batch in range(batch_size):
        start_index = random.randrange(0, len(benign_stream) - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = benign_stream[start_index : end_index]
        input_stream[batch] = torch.as_tensor(chunk[:-1])
        target_stream[batch] = torch.as_tensor(chunk[1:])
    input_stream = torch.LongTensor(input_stream).to(device)
    target_stream = torch.LongTensor(target_stream).to(device)
    return input_stream, target_stream   